In [2]:
import numpy as np
from stl import mesh
from sklearn.neighbors import KDTree
from collections import defaultdict
stl_file = "space_station/meshes/base_link.STL" 
ss_mesh = mesh.Mesh.from_file(stl_file)


Find unique vertices and their area-weighted normals.

In [3]:
unique_vertices = np.unique(ss_mesh.vectors.reshape(-1, 3), axis=0)
vertex_normals = defaultdict(list)

for i, (normal, triangle) in enumerate(zip(ss_mesh.normals, ss_mesh.vectors)):
    for vertex in triangle:
        vertex_tuple = tuple(vertex)  # Convert to tuple for defaultdict key
        if np.linalg.norm(normal) > 0.0:
            normal /= np.linalg.norm(normal)
        vertex_normals[vertex_tuple].append(normal*ss_mesh.areas[i])

# Calculate the average normal for each vertex
average_vertex_normals = {}
for vertex, normals in vertex_normals.items():
    #add the weighted contributions from the triangles surrounding each vertex
    average_normal = np.sum(normals, axis=0)
    # Normalize to divide by the sum of the weighted normals
    norm = np.linalg.norm(average_normal)
    if norm > 1e-6:  # Avoid division by zero
        average_vertex_normals[vertex] = average_normal / norm
    else:
        average_vertex_normals[vertex] = np.array([0.0, 0.0, 0.0])

# Convert the dictionaries to lists of (vertex, average_normal) if needed
vertex_normal_list = [(np.array(v), n) for v, n in average_vertex_normals.items()]


Use this to create viewpoints.

In [6]:
view_distance = 0.2
vps = [v + view_distance*n for v, n in average_vertex_normals.items()]
vps = np.array(vps)
#print(vps)

[[-5.20507038e-01  2.03938901e-01  2.48679668e-01]
 [-6.85978532e-01  1.16620392e-01 -3.77653167e-02]
 [-5.12115896e-01 -2.04612598e-01  2.47279257e-01]
 [-6.85469925e-01 -1.05695494e-01 -4.99430262e-02]
 [-5.10322332e-01 -2.04731047e-01  1.75066543e+00]
 [-6.85978532e-01 -1.16620392e-01  2.03776526e+00]
 [-5.06064534e-01  2.04902619e-01  1.75136459e+00]
 [-6.85469925e-01  1.05695494e-01  2.04994297e+00]
 [-6.37990177e-01 -2.80977845e-01 -3.28431204e-02]
 [-6.30143642e-01 -2.98159450e-01  2.03284311e+00]
 [-5.32986522e-01 -4.49338883e-01 -3.28431241e-02]
 [-5.20617187e-01 -4.63613868e-01  2.03284311e+00]
 [-3.84803414e-01 -5.81297159e-01 -3.28431204e-02]
 [-3.68913382e-01 -5.91509044e-01  2.03284311e+00]
 [-2.05445826e-01 -6.66162193e-01 -3.28431167e-02]
 [-1.87322438e-01 -6.71483696e-01  2.03284311e+00]
 [-9.57422983e-03 -6.99770689e-01  3.06151588e-17]
 [ 9.44425259e-03 -6.97058737e-01  2.03284311e+00]
 [ 1.87322438e-01 -6.71483696e-01 -3.28431167e-02]
 [ 2.05445826e-01 -6.66162193e-

In [13]:
def kd_filtering(viewpoints, normals):# --- Parameters ---
    spatial_threshold = 0.5    # Maximum distance for points to be considered "too close"
    normal_threshold = 0.1     # Maximum np.linalg.norm(np.cross(n1, n2)) for normals to be "too close"
    
    # --- Filtering Implementation ---
    filtered_points_list = []
    filtered_normals_list = []
    # Store the filtered points as a NumPy array for KD-Tree building
    # Initialize with an empty array of shape (0, 3)
    filtered_points_array = np.empty((0, 3))
    filtered_kdtree = None # KD-Tree will be built on filtered_points_array
    
    num_viewpoints = len(viewpoints)
    
    # Iterate through each original point
    for i in range(num_viewpoints):
        query_point = viewpoints[i]
        query_normal = normals[i]
    
        is_redundant = False # Assume we want to keep it until proven otherwise
    
        # Check against points ALREADY added to the filtered list
        if len(filtered_points_list) > 0:
            # Build KD-tree on the *currently filtered* points
            # This rebuilds the tree in each iteration, which is inefficient for large datasets
            # A better approach for large scale would be to use a structure that supports
            # incremental updates or a different sampling strategy.
            # However, for demonstrating the logic with KD-tree on filtered points, this works.
            filtered_kdtree = KDTree(filtered_points_array)
    
            # Query the KD-tree of *filtered* points for neighbors of the current point
            # within the spatial threshold.
            # query_point needs to be (1, 3)
            kept_neighbor_indices = filtered_kdtree.query_radius(query_point.reshape(1, -1), r=spatial_threshold)[0]
    
            # If there are any filtered points within the spatial radius
            if len(kept_neighbor_indices) > 0:
                # Now check the normal similarity against these spatially close *filtered* points
                for filtered_idx in kept_neighbor_indices:
                    kept_point = filtered_points_list[filtered_idx] # Get the point from the filtered list
                    kept_normal = filtered_normals_list[filtered_idx] # Get the normal from the filtered list
    
                    normal_angle_metric = np.linalg.norm(np.cross(query_normal, kept_normal))
    
                    if normal_angle_metric < normal_threshold:
                        # Found a spatially close *and* normal-similar point that was already kept
                        is_redundant = True
                        break # No need to check other kept neighbors for this point i
    
        # If the point is not redundant (i.e., not too close to any already kept point)
        if not is_redundant:
            # Add the current point and normal to the filtered lists
            filtered_points_list.append(query_point)
            filtered_normals_list.append(query_normal)
            # Update the array used for the next KD-tree build
            filtered_points_array = np.array(filtered_points_list)
    
    
    # Convert the final lists to NumPy arrays
    final_filtered_points = np.array(filtered_points_list)
    final_filtered_normals = np.array(filtered_normals_list)
    
    print(f"\nOriginal number of points: {num_viewpoints}")
    print(f"Filtered number of points: {len(final_filtered_points)}")
    print(f"Reduction: {num_viewpoints - len(final_filtered_points)}")
    
    # --- Returning the filtered points and normals ---
    return final_filtered_points, final_filtered_normals

In [14]:
#vertex_norms = np.array(average_vertex_normals.value
vertex_norms = [n for n in average_vertex_normals.values()]
pts, norms = kd_filtering(vps, vertex_norms)


Original number of points: 63
Filtered number of points: 56
Reduction: 7
